In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import gc
import os

In [18]:
user_id = 'idcc'
#user_id = 'nightofwings'

domain = 'https://www.pttweb.cc'
url = '/user/{}?t=message'.format(user_id)
output_path = '/Users/zoe/Desktop/ptt_crawler/github_data/'

usr_result = []
page_lst = []
replace_ele = ['                ', '\n', '    ']

In [19]:
def reply_parser(reply_soup):
    reply_lst = []
    for push in reply_soup.select('.e7-top'):
        push_content = push.select('.yellow--text')[-1].text.strip(':')
        #if cross year, 2019 > 2020, check month diff
        push_datetime_pre = push.select('.ml-3')[0].text
        push_floor = push.select('.e7-floor')[0].text
        push_tag = push.select('.mr-2')[-1].text
        reply_lst.append([push_content, push_datetime_pre, push_floor, push_tag])
    return pd.DataFrame(reply_lst, columns=['push_content', 'push_datetime_pre', 'push_floor', 'push_tag'])

In [20]:
def page_parser(page_soup):
    rs_lst = []
    for x in page_soup.select('.thread-item'):
        reply_df = reply_parser(x)
        #article info
        reply_df['article_date'] = x.select('.thread-posttime')[0].text
        reply_df['article_lnk'] = x.a['href']
        reply_df['article_title'] = x.a.text
        ath_text = x.select('.thread-list-author')[0].text
        for _e in replace_ele:
            ath_text = ath_text.replace(_e, '')
        reply_df['author'] = ath_text.split('- 發表於')[0][3:]
        #reply summary
        post_ele = x.select('.thread-popularity')[0].text.split(',')
        reply_df['art_all'] = post_ele[0].strip(' 留言')
        reply_df['art_count'] =  post_ele[1].strip(' 推噓總分: ')

        rs_lst.append(reply_df)
    return rs_lst

In [21]:
pg_cnt = 0

while True:
    try:
        rs = requests.get(domain + url)
    except Exception as e:
        print(e)
        time.sleep(60)
        continue
    
    soup = BeautifulSoup(rs.content)
    #parser
    cur_page = [x for x in soup.select('.e7-active')][0].text
    next_url = soup.find_all('a', text='下一頁')
    page_result = page_parser(soup)
    usr_result.extend(page_result)    
    #page record
    page_lst.append([cur_page, url])
    
    if pg_cnt % 50 == 0:
        print('current page is {}, url is {}'.format(pg_cnt, url))
        print(time.ctime())
    
    if not next_url:
        break
    
    url = next_url[0]['href']
    pg_cnt = pg_cnt + 1
    gc.collect()

current page is 0, url is /user/idcc?t=message
Sun Jan 12 17:45:47 2020


# transform result

In [22]:
rs_df = pd.concat(usr_result)
rs_df.replace(',', '，', regex=True, inplace=True)
rs_df.reset_index(drop=True, inplace=True)

rs_df_split = pd.concat([rs_df, rs_df.article_lnk.str.split('/', expand=True)[[2, 3]]], axis=1)
rs_df_split.rename({2:'board', 3:'article_id'}, axis=1, inplace=True)
rs_df_split['push_userid'] = user_id

# checking result

In [23]:
rs_df_split.groupby('push_tag').agg({'board':'count'})

,board
push_tag,
→,8
噓,4
推,21


# output

In [17]:
rs_df_split.to_csv(output_path + '{}_reply.csv'.format(user_id), index=False)

# verify

In [14]:
rs_df_split.groupby(['author', 'push_tag']).agg({'push_userid':'count'})

push_userid
author       push_tag             
AAAB         →                   2
             噓                   1
             推                   3
AAApower     →                   2
             推                   2
ACEP         推                   1
ADIE2        →                   2
AESA         →                   3
             推                   2
AHQJOE       推                   1
AIRJEREMYLIN 推                   1
AJCole       推                   1
ALLENWAN     →                   2
ANA1         →                   1
ASEgroup     →                   3
             噓                   3
             推                   3
Acalanatha   推                   1
AceGuard     推                   1
Adolf5566    →                   2
Adonisy      →                  15
             推                  27
AhSs         噓                   2
AhUtopian    →                  10
             推                  18
Ahhhhaaaa    推                   1
Airplane33   →                   3
             推                   1
AisinGioro   →                   2
             噓                   1
...                            ...
zsedcftgbloi →                   1
             推                   1
zsincerity   →                   7
             噓                   4
             推                  10
zsp40773     推                   1
ztopip       →                   3
             推                   6
zumaya1      推                   1
zxc25361     →                   1
zxc445567    →                   1
             噓                   1
             推                   3
zxc61062005  →                  20
             噓                   3
zxcelephant  →                  10
             噓                   4
             推                  14
zyquan1207   推                   1
zyxwvoklove  推                   2
zzahoward    →                  24
             推                  13
zzyyxx77     →                   2
             噓                   2
             推                   3
zzzz8931     推                   1
zzzzz533     推                   1
zzzzzzzzzzzy →                   2
             噓                   1
             推                   2

[4972 rows x 1 columns]

In [9]:
#next & candidate union > over 200 pages have some isuues...
#narrow down to boards...
#rs_df.shape

(12100, 12)

In [9]:
board
push_tag	
→	12253
噓	1510
推	14410

留言數量: 28172
送出的『推』: 14412
送出的『→』: 12250
送出的『噓』: 1510
    
push_tag	
→	8279
噓	1699
推	12212
rs_df.groupby('push_tag').agg({'author':'count'})

,author
push_tag,
→,8279
噓,1699
推,12212
